In [ ]:
import cohere
import numpy as np
import pandas as pd
import streamlit as st
from annoy import AnnoyIndex

# Access the API key value
api_key = 'E1vtwb38YrYiHwE4iNNkpRd8mH7nwUMzC622ZXIZ'

In [13]:
df = pd.read_csv('test-example-2021.csv')
df

,text,question,info_column_two,info_column_three,text_answer,info_column_five,info_column_six,info_column_seven,info_column_eight,info_column_9,10,11,12
0,QUAL-01,Does your product process protected health inf...,No,NaN,Responses to the questions in the HIPAA sectio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QUAL-02,Does the vended product host/support a mobile ...,Yes,NaN,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,QUAL-03,Will institution data be shared with or hosted...,Yes,Full Measure data centers are outsourced and h...,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,QUAL-04,Do you have a Business Continuity Plan (BCP)?,Yes,Please see attached document.,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,QUAL-05,Do you have a Disaster Recovery Plan (DRP)?,Yes,Please see attached document.,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,PCID-08,What payment processors/gateways does the syst...,NaN,NaN,Refer to PCI DSS Security Standards for supple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,PCID-09,Can the application be installed in a PCI DSS ...,NaN,NaN,Refer to PCI DSS Security Standards for supple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,PCID-10,Is the application listed as an approved PA-DS...,NaN,NaN,Refer to PCI DSS Security Standards for supple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,PCID-11,Does the system or products use a third party ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# add an id column
df['id'] = df.index

In [15]:
def chunk_text(df, width=1500, overlap=500):
    # create an empty dataframe to store the chunked text
    new_df = pd.DataFrame(columns=['id', 'text_chunk'])

    # iterate over each row in the original dataframe
    for index, row in df.iterrows():
        # split the text into chunks of size 'width', with overlap of 'overlap'
        chunks = []
        rows = []
        for i in range(0, len(row['text']), width - overlap):
            chunk = row['text'][i:i+width]
            chunks.append(chunk)

        # iterate over each chunk and add it to the new dataframe
        chunk_rows = []
        for i, chunk in enumerate(chunks):
            # calculate the start index based on the chunk index and overlap
            start_index = i * (width - overlap)

            # create a new row with the chunked text and the original row's ID
            new_row = {'id': row['id'], 'text_chunk': chunk, 'start_index': start_index}
            chunk_rows.append(new_row)
        chunk_df = pd.DataFrame(chunk_rows)
        new_df = pd.concat([new_df, chunk_df], ignore_index=True)

    return new_df



new_df = chunk_text(df)
# append text chunks to the original dataframe in id order
df = df.merge(new_df, on='id', how='left')
df


,text,question,info_column_two,info_column_three,text_answer,info_column_five,info_column_six,info_column_seven,info_column_eight,info_column_9,10,11,12,id,text_chunk,start_index
0,QUAL-01,Does your product process protected health inf...,No,NaN,Responses to the questions in the HIPAA sectio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,QUAL-01,0.0
1,QUAL-02,Does the vended product host/support a mobile ...,Yes,NaN,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,QUAL-02,0.0
2,QUAL-03,Will institution data be shared with or hosted...,Yes,Full Measure data centers are outsourced and h...,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,QUAL-03,0.0
3,QUAL-04,Do you have a Business Continuity Plan (BCP)?,Yes,Please see attached document.,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,QUAL-04,0.0
4,QUAL-05,Do you have a Disaster Recovery Plan (DRP)?,Yes,Please see attached document.,You are required to complete the questions in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,QUAL-05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,PCID-08,What payment processors/gateways does the syst...,NaN,NaN,Refer to PCI DSS Security Standards for supple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242,PCID-08,0.0
243,PCID-09,Can the application be installed in a PCI DSS ...,NaN,NaN,Refer to PCI DSS Security Standards for supple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,243,PCID-09,0.0
244,PCID-10,Is the application listed as an approved PA-DS...,NaN,NaN,Refer to PCI DSS Security Standards for supple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244,PCID-10,0.0
245,PCID-11,Does the system or products use a third party ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245,PCID-11,0.0


In [16]:

co = cohere.Client(api_key) 

# Get the embeddings
embeds = co.embed(texts=list(df['text_chunk']),
                  model="large",
                  truncate="RIGHT").embeddings
# Check the dimensions of the embeddings
embeds = np.array(embeds)
embeds.shape


(247, 4096)

In [17]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('search_index.ann')

True

In [18]:
# export the dataframe to a csv file
df.to_csv('hecvat_text_final.csv', index=False)